# Viterbi algo

In [2]:
def viterbi(observed_states_list, hidden_states, start_prob, transition_prob, emission_prob):
    dp = [{}]
    path = {}
 
    for y in hidden_states:
        dp[0][y] = start_prob[y] * emission_prob[y][observed_states_list[0]]
        path[y] = [y]

    for t in range(1, len(observed_states_list)):
        dp.append({})
        newpath = {}
 
        for y in hidden_states:
            max_prob = -1
            pos = -1
            for y0 in hidden_states:
                prob = dp[t - 1][y0] * transition_prob[y0][y] * emission_prob[y][observed_states_list[t]]
                if prob > max_prob:
                    max_prob = prob
                    pos = y0
            dp[t][y] = max_prob
            newpath[y] = path[pos] + [y]
 
        path = newpath
    (prob, state) = max((dp[t][y], y) for y in hidden_states)
    return path[state]

## TEST 1

In [3]:
hidden_states = (1, 2)
 
observations = 'OPOPOPOOPPPPPPPPPPOOOOOOOO'
start_probability = {1: 0.5, 2: 0.5}
 
transition_probability = {
   1 : {1: 0.8, 2: 0.2},
   2 : {1: 0.2, 2: 0.8}
}
 
emission_probability = {
   1 : {'O': 0.5, 'P': 0.5},
   2 : {'O': 0.1, 'P': 0.9}
}
print(viterbi(observations, hidden_states, start_probability, transition_probability, emission_probability))

[1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1]


## TEST 2

In [4]:
hidden_states = (1, 2)
 
observations = 'OPOPOPOOPPPPPPPPPPOOOOOOOO'
start_probability = {1: 0.5, 2: 0.5}
 
transition_probability = {
   1 : {1: 0.5, 2: 0.5},
   2 : {1: 0.5, 2: 0.5}
}
 
emission_probability = {
   1 : {'O': 0.5, 'P': 0.5},
   2 : {'O': 0.51, 'P': 0.49}
}
print(viterbi(observations, hidden_states, start_probability, transition_probability, emission_probability))

[2, 1, 2, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2]


# Forward-Backward

In [5]:
def get_alpha(observed_states_list, hidden_states, start_prob, transition_prob, emission_prob):
    fwd = []
    current_fwd = {}
    for s in hidden_states:
        current_fwd[s] = start_prob[s] * emission_prob[s][observed_states_list[0]]
    fwd.append(current_fwd)
    for t in range(1, len(observed_states_list)):
        current_fwd = {}
        for s in hidden_states:
            sum_alpha = 0
            for s0 in hidden_states:
                sum_alpha += fwd[t - 1][s0] *  transition_prob[s0][s] * emission_prob[s][observed_states_list[t]]
            current_fwd[s] = sum_alpha
        fwd.append(current_fwd)
    return fwd

def get_beta(observed_states_list, hidden_states, start_prob, transition_prob, emission_prob):
    bkw = []
    T = len(observed_states_list)
    current_bkw = {}
    for s in hidden_states:
        current_bkw[s] = 1
    bkw.append(current_bkw)
    for t in range(1, len(observed_states_list)):
        current_bkw = {}
        for s in hidden_states:
            sum_beta = 0
            for s0 in hidden_states:
                sum_beta += bkw[t - 1][s0] *  transition_prob[s][s0] * emission_prob[s0][observed_states_list[T - t]]
            current_bkw[s] = sum_beta
        bkw.append(current_bkw)
    bkw.reverse()
    return bkw

def forward_backward(observed_states_list, hidden_states, start_prob, transition_prob, emission_prob):
    alpha = get_alpha(observed_states_list, hidden_states, start_prob, transition_prob, emission_prob) 
    beta = get_beta(observed_states_list, hidden_states, start_prob, transition_prob, emission_prob)
    answer = []
    for s in hidden_states:
        answer_row = []
        for t in range(len(observed_states_list)):
            answer_row.append(alpha[t][s] * beta[t][s] / sum(alpha[-1].values()))
        answer.append(answer_row)
    return answer

## TEST 1

In [6]:
hidden_states = (1, 2)
 
observations = 'OPOPOPOOPPPPPPPPPPOOOOOOOO'
start_probability = {1: 0.5, 2: 0.5}
 
transition_probability = {
   1 : {1: 0.8, 2: 0.2},
   2 : {1: 0.2, 2: 0.8}
}
 
emission_probability = {
   1 : {'O': 0.5, 'P': 0.5},
   2 : {'O': 0.1, 'P': 0.9}
}
result = forward_backward(observations, hidden_states, start_probability, transition_probability, emission_probability)
for a, b in zip(*result):
    print('{:.2f}\t{:.2f}'.format(a, b))

0.86	0.14
0.77	0.23
0.88	0.12
0.78	0.22
0.89	0.11
0.80	0.20
0.91	0.09
0.86	0.14
0.46	0.54
0.27	0.73
0.19	0.81
0.15	0.85
0.14	0.86
0.14	0.86
0.15	0.85
0.19	0.81
0.28	0.72
0.47	0.53
0.89	0.11
0.96	0.04
0.98	0.02
0.98	0.02
0.98	0.02
0.98	0.02
0.97	0.03
0.94	0.06


## TEST 2

In [7]:
hidden_states = (1, 2)
 
observations = 'OPOPOPOOPPPPPPPPPPOOOOOOOO'
start_probability = {1: 0.5, 2: 0.5}
 
transition_probability = {
   1 : {1: 0.5, 2: 0.5},
   2 : {1: 0.5, 2: 0.5}
}
 
emission_probability = {
   1 : {'O': 0.5, 'P': 0.5},
   2 : {'O': 0.51, 'P': 0.49}
}
result = forward_backward(observations, hidden_states, start_probability, transition_probability, emission_probability)
for a, b in zip(*result):
    print('{:.2f}\t{:.2f}'.format(a, b))

0.50	0.50
0.51	0.49
0.50	0.50
0.51	0.49
0.50	0.50
0.51	0.49
0.50	0.50
0.50	0.50
0.51	0.49
0.51	0.49
0.51	0.49
0.51	0.49
0.51	0.49
0.51	0.49
0.51	0.49
0.51	0.49
0.51	0.49
0.51	0.49
0.50	0.50
0.50	0.50
0.50	0.50
0.50	0.50
0.50	0.50
0.50	0.50
0.50	0.50
0.50	0.50
